In [84]:
# from llama_index import download_loader
from langchain.document_loaders import PyPDFLoader, SeleniumURLLoader
from langchain.chains.question_answering import load_qa_chain
from docx import Document
from pathlib import Path
from pprint import pprint
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import pdfplumber
import openai
from openai import OpenAI
from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

load_dotenv()

True

In [26]:
# PDFReader = download_loader("PDFReader")
# loader = PDFReader()
# resume = loader.load_data(file=Path("/Users/alexmcgraw/Documents/Resumes:Cover Letters:Case_stuff/Alex McGraw Resume, December '23.pdf"))

NameError: name 'download_loader' is not defined

In [2]:
lang_PDFReader = pdfplumber.open("/Users/alexmcgraw/Documents/Resumes:Cover Letters:Case_stuff/Alex McGraw Resume, December '23, v2.pdf")
lang_PDFReader_page = lang_PDFReader.pages[0]
resume = lang_PDFReader_page.extract_text()
# resume = lang_PDFReader.load()
pprint(resume)

('ALEX MCGRAW\n'
 'Jamcgraw21@gmail.com\n'
 'github.com/JAlexMcGraw www.linkedin.com/in/josephalexmcgraw (972) 768-3879\n'
 'EXPERIENCE\n'
 'Slalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois '
 'January 2023 - Present\n'
 '● Scope a generative AI project for a major pharmaceuticals company, to help '
 'multiple finance teams with developing\n'
 'presentation support material for the C-suite, and Q&A over hundreds of '
 'various financial documents\n'
 '● Build a generative AI POC using modified Langchain and Llama-index code, '
 'OpenAI models, and Azure platform, for\n'
 'users to query a corpus of hundreds of source documents and receive insights '
 'from various economic reports\n'
 '● Analyze data discrepancies between the client’s legacy call routing tool '
 'and the newly implemented Salesforce tool\n'
 'using SQL, to ensure that the data migration between the tools is a '
 'one-to-one match\n'
 'Slalom Consulting – Associate Consultant (Data Science/En

In [85]:
import re

def split_string_by_headers(input_string):
    # Define a regular expression pattern to match all upper case words
    header_pattern = re.compile(r'\n\b[A-Z]+\b')

    # Find all matches of the pattern in the input string
    header_matches = header_pattern.finditer(input_string)

    # Extract the indices of the matches
    header_indices = [match.start() for match in header_matches]

    # Add the start and end indices of the string to mark the boundaries
    header_indices = [-1] + header_indices + [len(input_string)]

    # Split the string based on the identified header indices
    result_parts = [input_string[header_indices[i] + 1:header_indices[i + 1]].strip() for i in range(len(header_indices) - 1)]

    return result_parts

# Example usage:
input_string = """
HEADER1
This is the content under HEADER1.

SUBHEADER1
Content under SUBHEADER1.

HEADER2
This is the content under HEADER2.

Bullet Points:
- Point 1
- Point 2

HEADER3
Content under HEADER3.
"""

result = split_string_by_headers(input_string)

for i, part in enumerate(result, start=1):
    print(f"Part {i}:\n{part}\n{'=' * 40}")


Part 1:
HEADER1
This is the content under HEADER1.

SUBHEADER1
Content under SUBHEADER1.

HEADER2
This is the content under HEADER2.

Bullet Points:
- Point 1
- Point 2

HEADER3
Content under HEADER3.


In [4]:
split_resume = split_string_by_headers(resume)
split_resume_no_header = split_resume[1:]
for i, part in enumerate(split_resume_no_header, start=0):
    print(f"Part {i}:\n{part}\n{'=' * 40}")

Part 0:
EXPERIENCE
Slalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois January 2023 - Present
● Scope a generative AI project for a major pharmaceuticals company, to help multiple finance teams with developing
presentation support material for the C-suite, and Q&A over hundreds of various financial documents
● Build a generative AI POC using modified Langchain and Llama-index code, OpenAI models, and Azure platform, for
users to query a corpus of hundreds of source documents and receive insights from various economic reports
● Analyze data discrepancies between the client’s legacy call routing tool and the newly implemented Salesforce tool
using SQL, to ensure that the data migration between the tools is a one-to-one match
Slalom Consulting – Associate Consultant (Data Science/Engineering); Chicago, Illinois June 2021 - January 2023
● Wrote, tested, and productionalized 11 machine learning models alongside two other data scientists using Azure for a
Fortune 50 

In [5]:
# SimpleWebPageReader = download_loader("SimpleWebPageReader")
# web_page_loader = SimpleWebPageReader()
# web_doc = web_page_loader.load_data(urls=["https://jobs.lever.co/cohere/105c8f72-c0ba-4f23-bc65-83de9bcd70d2"])

In [5]:
lang_web_loader = SeleniumURLLoader(["https://boards.greenhouse.io/snorkelai/jobs/4971098004"])
web_doc = lang_web_loader.load()

In [6]:
pprint(web_doc[0].page_content)

('Apply Now\n'
 '\n'
 'Applied Machine Learning Engineer\n'
 '\n'
 'at Snorkel AI\n'
 '\n'
 '(View all jobs)\n'
 '\n'
 'Remote - US Only\n'
 '\n'
 'At Snorkel AI, we’re redefining how people and organizations build AI '
 'applications. Snorkel started as a research project in the Stanford AI Lab '
 'in 2016, creating a higher-level interface to machine learning through '
 'programmatically labeled and managed training data. From deploying in some '
 'of the world’s largest and most sophisticated tech organizations, to '
 'empowering scientists, doctors, and journalists — we’ve seen firsthand how '
 'this approach democratizes and accelerates AI. Now, we’re building Snorkel '
 'Flow to bring our technology to everyone!\n'
 '\n'
 'Excited to help us redefine how AI applications are built? Apply to be the '
 'newest Snorkeler!\n'
 '\n'
 'As an Applied Machine Learning Engineer, you’ll research and utilize '
 'established and state-of-the-art machine learning (ML) techniques to '
 "success

In [7]:
def extract_text_with_bs(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    text_content = soup.get_text(separator=' ', strip=True)
    return text_content

# Example usage
html_string = '<p>This is <b>sample</b> HTML <a href="#">text</a>.</p>'
text_content = extract_text_with_bs(html_string)
print(text_content)


This is sample HTML text .


In [8]:
job_doc = extract_text_with_bs(web_doc[0].page_content)
pprint(job_doc)

('Apply Now\n'
 '\n'
 'Applied Machine Learning Engineer\n'
 '\n'
 'at Snorkel AI\n'
 '\n'
 '(View all jobs)\n'
 '\n'
 'Remote - US Only\n'
 '\n'
 'At Snorkel AI, we’re redefining how people and organizations build AI '
 'applications. Snorkel started as a research project in the Stanford AI Lab '
 'in 2016, creating a higher-level interface to machine learning through '
 'programmatically labeled and managed training data. From deploying in some '
 'of the world’s largest and most sophisticated tech organizations, to '
 'empowering scientists, doctors, and journalists — we’ve seen firsthand how '
 'this approach democratizes and accelerates AI. Now, we’re building Snorkel '
 'Flow to bring our technology to everyone!\n'
 '\n'
 'Excited to help us redefine how AI applications are built? Apply to be the '
 'newest Snorkeler!\n'
 '\n'
 'As an Applied Machine Learning Engineer, you’ll research and utilize '
 'established and state-of-the-art machine learning (ML) techniques to '
 "success

### Using Langchain

In [9]:
chat = ChatOpenAI(temperature=0.5)

In [60]:
[doc for doc in split_resume_no_header]

['EXPERIENCE\nSlalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois January 2023 - Present\n● Scope a generative AI project for a major pharmaceuticals company, to help multiple finance teams with developing\npresentation support material for the C-suite, and Q&A over hundreds of various financial documents\n● Build a generative AI model using the Langchain and Llama-index frameworks, OpenAI models, and Azure platform, for\nusers to query a corpus of hundreds of source documents and receive insights from various economic reports\n● Analyze data discrepancies between the client’s legacy call routing tool and the newly implemented Salesforce tool\nusing SQL, to ensure that the data migration between the tools is a one-to-one match\nSlalom Consulting – Associate Consultant (Data Science/Engineering); Chicago, Illinois June 2021 - January 2023\n● Wrote, tested, and productionalized 11 machine learning models alongside two other data scientists using Azure for a\nFort

In [68]:
messages = [
    SystemMessage(
        content="""You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """
    ),
    HumanMessage(
        content=f"RESUME: {[doc for doc in split_resume_no_header]}\n-----\nJOB POSTING: {job_doc}"
    )
]

In [ ]:
# sys_prompt_template = """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
#             will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
#             Make sure that experiences from the resume tie back to specific points from the job posting.
#             Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
#             specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
#             For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
#             try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
#             discuss what skills you have developed and connect these back to the target role.
#             For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
#             Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
#             do not exceed a page in length and do not make any information up.
#             -----------
#             RESUME: {split_resume_no_header}
#             JOB POSTING: {job_doc}"""
            
# prompt = PromptTemplate(
#     template=sys_prompt_template,
#     input_variables=["split_resume_no_header", "job_doc"]
# )       

In [69]:
output = chat(messages)

In [72]:
output.content

'Dear Hiring Manager,\n\nI am writing to apply for the position of Member of Technical Staff, Data Pipelines at Cohere. With my strong background in data science and engineering, I believe I have the skills and experience necessary to contribute to the success of your team.\n\nIn my current role as a Consultant at Slalom Consulting, I have gained extensive experience in working with large datasets and implementing innovative techniques for data ingestion and integration. I have successfully built generative AI models using frameworks such as Langchain and Llama-index, and have utilized Azure platform for data analysis and insights generation. These experiences align closely with the responsibilities outlined in the job posting, particularly in terms of evaluating data impact on models and continuously improving data quality.\n\nAdditionally, I have a proven track record of collaborating with cross-functional teams and effectively communicating complex ideas. As an Associate Consultant 

In [73]:
document = Document()
document.add_paragraph(output.content)

In [74]:
document.save('cover_letter.docx')

### Trying to implement the same thing above ^^, but using the prompt templates, because don't think the messages used above are the right way to do it. Also, want to be able to allow an optional input for extrac context from the user.

In [ ]:
messages = [
    SystemMessagePromptTemplate.from_template(
            """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            -----------
            RESUME: {split_resume_no_header}
            JOB POSTING: {job_doc}
            """
    ),
    HumanMessagePromptTemplate.from_template(
            "{extra_context}"
    )
]

chat_prompt = ChatPromptTemplate.from_messages(messages)

In [78]:
optional_text = ""

In [79]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Create a list of messages
messages = [
    {
        "role": "system",
        "content": """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            
            "RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"""
    },
    {
        "role": "user",
        "content": """"""
    }
    ("system", """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            
            "RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"""),
    ("document_chunks", split_resume_no_header),
    ("document", job_doc),
    ("optional_text", optional_text),
]

# Create a prompt template from the list of messages
prompt = ChatPromptTemplate.from_messages(messages)

# Instantiate the ChatOpenAI class with the desired model
model = ChatOpenAI(model="gpt-3.5-turbo-16k")

# Chain the prompt template and the ChatOpenAI model together
chain = prompt | model | StrOutputParser()

# Invoke the chain with the desired input
for chunk in chain.invoke(system_message):
    print(chunk, end="", flush=True)


ValueError: Unexpected message type: document_chunks. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.

In [103]:
split_resume_no_header

['EXPERIENCE\nSlalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois January 2023 - Present\n● Scope a generative AI project for a major pharmaceuticals company, to help multiple finance teams with developing\npresentation support material for the C-suite, and Q&A over hundreds of various financial documents\n● Build a generative AI POC using modified Langchain and Llama-index code, OpenAI models, and Azure platform, for\nusers to query a corpus of hundreds of source documents and receive insights from various economic reports\n● Analyze data discrepancies between the client’s legacy call routing tool and the newly implemented Salesforce tool\nusing SQL, to ensure that the data migration between the tools is a one-to-one match\nSlalom Consulting – Associate Consultant (Data Science/Engineering); Chicago, Illinois June 2021 - January 2023\n● Wrote, tested, and productionalized 11 machine learning models alongside two other data scientists using Azure for a\nFortune

### Prompting only for the body doesn't seem to work.

In [21]:
body_prompt = """
You are a job application assistant. You are to write the body of a cover letter, not the introduction nor conclusion, by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
Make sure that experiences from the resume tie back to specific points from the job posting.
"""

body_messages = [
    {
        "role":"system",
        "content":body_prompt
    },
    {
        "role":"user",
        "content":f"RESUME: {split_resume_no_header} ----- JOB POSTING: {job_doc}"
    }
]

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=body_messages,
    temperature=0.7
)

In [13]:
dict(response).keys()

dict_keys(['id', 'choices', 'created', 'model', 'object', 'system_fingerprint', 'usage'])

In [22]:
pprint(response.choices[0].message.content)

('Dear Hiring Manager,\n'
 '\n'
 'I am writing to apply for the position of Applied Machine Learning Engineer '
 'at Snorkel AI. I am excited about the opportunity to work with a company '
 'that is redefining how people and organizations build AI applications. With '
 'my experience in data science and engineering, I believe I would be a '
 'valuable addition to your team.\n'
 '\n'
 'In my current role as a Consultant at Slalom Consulting, I have been '
 'involved in a range of projects that align with the responsibilities '
 'outlined in the job posting. For example, I have scoped and built generative '
 'AI projects using OpenAI models and Azure platform, which involved '
 'developing solutions to address specific business needs. These projects '
 'required me to engage with customers, present findings and recommendations, '
 'and deliver business impact through the deployment of machine learning '
 'models.\n'
 '\n'
 'Additionally, I have experience in designing, developing, and de

In [15]:
job_doc

'Apply Now\n\nApplied Machine Learning Engineer\n\nat Snorkel AI\n\n(View all jobs)\n\nRemote - US Only\n\nAt Snorkel AI, we’re redefining how people and organizations build AI applications. Snorkel started as a research project in the Stanford AI Lab in 2016, creating a higher-level interface to machine learning through programmatically labeled and managed training data. From deploying in some of the world’s largest and most sophisticated tech organizations, to empowering scientists, doctors, and journalists — we’ve seen firsthand how this approach democratizes and accelerates AI. Now, we’re building Snorkel Flow to bring our technology to everyone!\n\nExcited to help us redefine how AI applications are built? Apply to be the newest Snorkeler!\n\nAs an Applied Machine Learning Engineer, you’ll research and utilize established and state-of-the-art machine learning (ML) techniques to successfully deliver solutions to our customers. You\'ll engage new ML use cases through hands-on custom

### Testing again, with recommendations from https://community.openai.com/t/prompt-model-with-list-of-strings-and-ranked-importance/565708/3 . Write whole cover letter, reranking resume first and/or recursively building cover letter.

Identify most important parts of a job listing

In [16]:
import re

job_doc_no_extra_space = re.sub(' +', ' ', job_doc)

In [18]:
job_doc_no_extra_space

'Apply Now\n\nApplied Machine Learning Engineer\n\nat Snorkel AI\n\n(View all jobs)\n\nRemote - US Only\n\nAt Snorkel AI, we’re redefining how people and organizations build AI applications. Snorkel started as a research project in the Stanford AI Lab in 2016, creating a higher-level interface to machine learning through programmatically labeled and managed training data. From deploying in some of the world’s largest and most sophisticated tech organizations, to empowering scientists, doctors, and journalists — we’ve seen firsthand how this approach democratizes and accelerates AI. Now, we’re building Snorkel Flow to bring our technology to everyone!\n\nExcited to help us redefine how AI applications are built? Apply to be the newest Snorkeler!\n\nAs an Applied Machine Learning Engineer, you’ll research and utilize established and state-of-the-art machine learning (ML) techniques to successfully deliver solutions to our customers. You\'ll engage new ML use cases through hands-on custom

In [21]:
job_importance_prompt = """You are a job search assistant. You are tasked with identifying the top 3 most important elements of a job listing in descending order, along with all of the info in the listing that accompanies the element. 
                            Return your response in JSON format, where each key is a number between 1-5, inclusive, which represents its ranking with 1 being first most important, and 3 being third most important.
                            The values returned should be a list, where the first element in the list (index 0) should be the exact element from the text, and the second element in the list (index 1) should be the info accompanying the element.
                            The job posting will come after "JOB POSTING:" """

body_messages = [
    {
    "role": "system",
    "content": job_importance_prompt
    },
    {
    "role": "user",
    "content": f"JOB POSTING: {job_doc_no_extra_space}"
    }
]

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=body_messages,
    temperature=0.5
)
    

In [22]:
pprint(response.choices[0].message.content)

('{\n'
 '  "1": [\n'
 '    "Job Title",\n'
 '    "Applied Machine Learning Engineer"\n'
 '  ],\n'
 '  "2": [\n'
 '    "Main Responsibilities",\n'
 '    "Assist customers with the delivery of a Machine Learning project from '
 'beginning to end, including business case definition and scoping, data '
 'aggregation and exploration, algorithm selection, and model deployment to '
 'deliver business impact to the organization"\n'
 '  ],\n'
 '  "3": [\n'
 '    "Preferred Qualifications",\n'
 '    "5+ years of professional experience with machine learning, or 2+ years '
 'of professional experience with machine learning with an advanced degree in '
 'a relevant field"\n'
 '  ]\n'
 '}')


#### Doing the "pick elements" thing isn't quite working as I wanted, so now just gonna make a baseline output using the same prompts as I used during Langchain usage. Wanna start testing this all out with some Pydantic after

In [28]:
system_message = """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """

user_message = f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"

body_messages = [
    {
        "role":"system",
        "content":system_message
    },
    {
        "role":"user",
        "content":user_message
    }
]

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=body_messages,
    temperature=0.7
)

In [ ]:
pprint(response.choices[0].message.content)

In [ ]:
lang_PDFReader = pdfplumber.open("/Users/alexmcgraw/Documents/Resumes:Cover Letters:Case_stuff/Alex McGraw Resume, December '23, v2.pdf")
lang_PDFReader_page = lang_PDFReader.pages[0]
resume = lang_PDFReader_page.extract_text()
# resume = lang_PDFReader.load()
pprint(resume)

In [86]:
from typing import ClassVar, Optional

class CoverLetterGenerator(BaseModel):

    resume_path: str
    job_posting_url:str
    default_sys_prompt: ClassVar[str]="""You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """
    default_user_prompt: ClassVar[str]=f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"

    # def __init__(self, resume_path: str, job_posting_url: str, default_sys_prompt: Optional[str]=None, default_user_prompt: Optional[str]=None):
    #     super().__init__()
    #     self.resume_path=resume_path
    #     self.job_posting_url=job_posting_url
    #     self.default_sys_prompt=default_sys_prompt or self.default_sys_prompt
    #     self.default_user_prompt=default_user_prompt or self.default_user_prompt
    
    def _read_resume(self) -> str:
        read_in_resume = pdfplumber.open(self.resume_path)
        pages = read_in_resume.pages[0]
        resume = pages.extract_text()

        return resume
    
    def _split_resume_into_list(self) -> list:

        resume = self._read_resume()
        # Define a regular expression pattern to match all upper case words
        header_pattern = re.compile(r'\n\b[A-Z]+\b')

        # Find all matches of the pattern in the input string
        header_matches = header_pattern.finditer(resume)

        # Extract the indices of the matches
        header_indices = [match.start() for match in header_matches]

        # Add the start and end indices of the string to mark the boundaries
        header_indices = [-1] + header_indices + [len(resume)]

        # Split the string based on the identified header indices
        result_parts = [resume[header_indices[i] + 1:header_indices[i + 1]].strip() for i in range(len(header_indices) - 1)]

        return result_parts[1:]
    
    def _load_job_listing_html(self) -> str:
        from langchain.document_loaders import SeleniumURLLoader

        web_doc = SeleniumURLLoader([self.job_posting_url]).load()

        return web_doc
    
    def _load_job_listing_text(self) -> str:
        from bs4 import BeautifulSoup

        html = self._load_job_listing_html()
        soup = BeautifulSoup(html[0].page_content, 'html.parser')
        text_content = soup.get_text(separator=' ', strip=True)
        
        return text_content
    
    def generate_cover_letter(self, temperature: float = 0.7) -> str:
        from openai import OpenAI

        split_resume_no_header = self._split_resume_into_list()
        job_doc = self._load_job_listing_text()

        body_messages = [
            {
                "role":"system",
                "content":self.default_sys_prompt
            },
            {
                "role":"user",
                "content":self.default_user_prompt
            }
        ]

        client = OpenAI()

        response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=body_messages,
            temperature=temperature
        )

        return response.choices[0].message.content

In [90]:
class CoverLetterGenerator(BaseModel):

    resume_path: str
    job_posting_url:str
    default_sys_prompt: ClassVar[str]="""You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """
    # default_user_prompt: ClassVar[str]=f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"
    default_user_prompt: str = ""

    # def __init__(self, resume_path: str, job_posting_url: str, default_sys_prompt: Optional[str]=None, default_user_prompt: Optional[str]=None):
    #     super().__init__()
    #     self.resume_path=resume_path
    #     self.job_posting_url=job_posting_url
    #     self.default_sys_prompt=default_sys_prompt or self.default_sys_prompt
    #     self.default_user_prompt=default_user_prompt or self.default_user_prompt

    def __init__(self, resume_path: str, job_posting_url: str):
        super().__init__(resume_path=resume_path, job_posting_url=job_posting_url)
        self.set_default_user_prompt()

    def set_default_user_prompt(self):
        split_resume_no_header = self._split_resume_into_list()
        job_doc = self._load_job_listing_text()
        self.default_user_prompt = f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"

    
    def _read_resume(self) -> str:
        import pdfplumber 

        read_in_resume = pdfplumber.open(self.resume_path)
        pages = read_in_resume.pages[0]
        resume = pages.extract_text()

        return resume
    
    def _split_resume_into_list(self) -> list:

        resume = self._read_resume()
        # Define a regular expression pattern to match all upper case words
        header_pattern = re.compile(r'\n\b[A-Z]+\b')

        # Find all matches of the pattern in the input string
        header_matches = header_pattern.finditer(resume)

        # Extract the indices of the matches
        header_indices = [match.start() for match in header_matches]

        # Add the start and end indices of the string to mark the boundaries
        header_indices = [-1] + header_indices + [len(resume)]

        # Split the string based on the identified header indices
        result_parts = [resume[header_indices[i] + 1:header_indices[i + 1]].strip() for i in range(len(header_indices) - 1)]

        return result_parts[1:]
    
    def _load_job_listing_html(self) -> str:
        from langchain.document_loaders import SeleniumURLLoader

        web_doc = SeleniumURLLoader([self.job_posting_url]).load()

        return web_doc
    
    def _load_job_listing_text(self) -> str:
        from bs4 import BeautifulSoup

        html = self._load_job_listing_html()
        soup = BeautifulSoup(html[0].page_content, 'html.parser')
        text_content = soup.get_text(separator=' ', strip=True)
        
        return text_content
    
    def generate_cover_letter(self, temperature: float = 0.7) -> str:
        from openai import OpenAI

        split_resume_no_header = self._split_resume_into_list()
        job_doc = self._load_job_listing_text()

        body_messages = [
            {
                "role":"system",
                "content":self.default_sys_prompt
            },
            {
                "role":"user",
                "content":self.default_user_prompt
            }
        ]

        client = OpenAI()

        response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=body_messages,
            temperature=temperature
        )

        return response.choices[0].message.content


In [91]:
cl = CoverLetterGenerator(resume_path="/Users/alexmcgraw/Documents/Resumes:Cover Letters:Case_stuff/Alex McGraw Resume, December '23, v2.pdf", 
                          job_posting_url='https://boards.greenhouse.io/snorkelai/jobs/4971098004')
cover_letter = cl.generate_cover_letter()

In [92]:
pprint(cover_letter)

('Dear Hiring Manager,\n'
 '\n'
 'I am writing to express my strong interest in the position of Applied '
 'Machine Learning Engineer at Snorkel AI. With my background in data science '
 'and engineering, along with my experience in delivering machine learning '
 'solutions to clients, I believe I would be a valuable addition to your '
 'team.\n'
 '\n'
 'After reviewing the job posting, I am particularly drawn to the opportunity '
 'to research and utilize established and state-of-the-art machine learning '
 'techniques to deliver solutions to customers. My experience at Slalom '
 'Consulting has allowed me to work on a variety of machine learning projects, '
 'including building generative AI models for financial analysis and '
 'developing machine learning models for churn risk prediction. These '
 'experiences have equipped me with a deep understanding of machine learning '
 'frameworks and technologies, such as PyTorch, Transformers, Scikit-learn, '
 'NumPy, and Pandas, which are m